In [17]:
# all the imports
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import pandas as pd
import csv

In [4]:
curr_dir = Path.cwd()
data_dir = curr_dir / 'User_data'
log_file_path = []

# get the latest log file
for user in data_dir.iterdir():
    for session in user.iterdir():
        for log in session.iterdir():
            if log.suffix == '.log':
                log_file_path.append(log)


In [5]:
file_path = log_file_path[0]
print(file_path)

/mnt/c/Users/Abhishek/Desktop/OpenBCI_DataAnlaysis/User_data/12345/session_1/12345_session_1_22-03-2023_21-53.log


In [14]:
#************************************TRACKING MODULE**********************************
def getTrackScore(track_list,file_pointer):

    split_list = []
    count = 0
    i=0
    total_time_out = []
    mean_deviation_list = []

    while i <len(track_list):
        if track_list[i] != 'AUTO\n':
            j = i
            temp_list=[]
            while(j<len(track_list) and track_list[j]!='AUTO\n'):
                temp_list.append(track_list[j])
                j+=1
            split_list.append(temp_list)
            i = j
        i+=1

    for event in split_list:
        # print(type(event[0]))
        temp_mean =0.0
        time_in = 0
        time_out = 0
        for i in range(len(event)):
            event_dict = eval(event[i])
            if(i==0):
                time_in = event_dict['total']['time_out_ms']
            if(i==len(event)-1):
                time_out = event_dict['total']['time_out_ms']
            temp_mean += event_dict['total']['deviation_mean']

        mean_deviation_list.append(temp_mean/len(event))
        total_time_out.append(time_out-time_in)
        # print(time_out-time_in)

    #write to file
    file_pointer.write('TRACKING MODULE\n')
    file_pointer.write('Mean Deviation: '+str(mean_deviation_list)+'\n')
    file_pointer.write('Total Time Out: '+str(total_time_out)+'\n')
    file_pointer.write('END\n')
    # print('Mean Deviation: ',mean_deviation_list)
    # print('Total Time Out: ',total_time_out)
    # # [print(ele) for ele in split_list]

#*************************************************************************************

In [15]:
#************************************SYSMON MODULE************************************
def getSysmonScore(sysmon_list,time_dict,file_pointer):
    total_failures = 0
    total_hits = 0
    false_alarm = 0
    response_time = 0

    for i,event in enumerate(sysmon_list)   :
        if event[5] == 'FAILURE\n':
            total_failures += 1
            #check if the next event is a hit
            if(sysmon_list[i+1][5]=='HIT\n'):
                total_hits += 1

            #check if the next event is a miss
            if(sysmon_list[i+1][5]=='MISS\n'):
                #get the timestamp
                time1 = sysmon_list[i+1][0].split('.')[0]
                time_dict[time1] = 1
                false_alarm += 1

            #calculate response time
            time1 = datetime.strptime(sysmon_list[i][0], '%H:%M:%S.%f')
            time2 = datetime.strptime(sysmon_list[i+1][0], '%H:%M:%S.%f')
            res = time2 - time1
            # print(res.total_seconds())
            response_time += res.total_seconds()

    #write to file
    avg_response_time = response_time/total_failures
    file_pointer.write('SYSMON MODULE\n')
    file_pointer.write('Total Failures: '+str(total_failures)+'\n')
    file_pointer.write('Total Hits: '+str(total_hits)+'\n')
    file_pointer.write('False Alarms: '+str(false_alarm)+'\n')
    file_pointer.write('Average Response Time: '+str(avg_response_time)+'\n')
    file_pointer.write('END'+'\n')
    # print('Total Failures: ',total_failures)
    # print('Total Hits: ',total_hits)
    # print('False Alarms: ',false_alarm)
    # print('Average Response Time: ',avg_response_time)

#*************************************************************************************

In [16]:
#************************************COMMUNICATION MODULE************************************
def getCommScore(comm_list,time_dict,file_pointer):
    #first split on TARGET
    split_list = []
    response_times = []
    repsonse_accuracies = []
    i=0
    while i < len(comm_list):
        if comm_list[i][6] == 'TARGET':
            j = i+1
            while j < len(comm_list) and comm_list[j][6] != 'TARGET':
                j+=1
            split_list.append(comm_list[i:j])
            i = j
            continue
        i+=1

    #traverse through the split list and get the score
    for event in split_list:
        channel =''
        #case when the prompt is for the subject
        if event[0][4] == 'OWN':
            channel = event[0][5]
            curr_target = event[0][7].strip()
            #when ENTER is not pressed
            if event[-1][5] != 'RETURN':
                #add 30 seconds to the response time
                #add 0 to the accuracy
                response_times.append(30)
                repsonse_accuracies.append(0)

                #get the start time
                start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
                start_time = start_time.strftime('%H:%M:%S')
                #get the end time
                end_time = datetime.strptime(event[-1][0], '%H:%M:%S.%f')
                end_time = end_time.strftime('%H:%M:%S')

                #update the time_dict
                for time in time_dict:
                    if time >= start_time and time <= end_time:
                        time_dict[time] = 1
                continue

            #when ENTER is pressed
            #get the last set frequency
            last_freq = event[-2][6].strip()
            if(curr_target == last_freq):
                #correct frequency is set
                repsonse_accuracies.append(1)
                start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
                end_time = datetime.strptime(event[-2][0], '%H:%M:%S.%f')
                response_time = end_time - start_time
                response_times.append(response_time.total_seconds())
                continue

            #correct frequency is not set
            error = abs(float(curr_target) - float(last_freq))/float(curr_target)
            accuracy = 1 - error
            repsonse_accuracies.append(accuracy)
            start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
            end_time = datetime.strptime(event[-2][0], '%H:%M:%S.%f')
            response_time = end_time - start_time
            response_times.append(response_time.total_seconds())
            start_time = start_time.strftime('%H:%M:%S')
            end_time = end_time.strftime('%H:%M:%S')
            #update the time_dicts
            for time in time_dict:
                if time >= start_time and time <= end_time:
                    time_dict[time] = 1


        #case when the prompt is not for the subject
        if event[0][4] == 'OTHER':
            channel = event[0][5]
            #check if the frequency is changed
            if(len(event)>=5):
                #frequency is changed
                response_times.append(30)
                repsonse_accuracies.append(0)
                #get the start time
                start_time = datetime.strptime(event[0][0], '%H:%M:%S.%f')
                start_time = start_time.strftime('%H:%M:%S')
                #get the end time
                end_time = datetime.strptime(event[-1][0], '%H:%M:%S.%f')
                end_time = end_time.strftime('%H:%M:%S')

                #update the time_dict
                for time in time_dict:
                    if time >= start_time and time <= end_time:
                        time_dict[time] = 1
            else:
                #frequency is not changed
                response_times.append(0)
                repsonse_accuracies.append(1)

    #write the response times and accuracies to the file
    file_pointer.write('Communication Module\n')
    file_pointer.write('Response Times: ')
    for time in response_times:
        file_pointer.write(str(time)+' ')
    file_pointer.write('\n')
    file_pointer.write('Response Accuracies: ')
    for accuracy in repsonse_accuracies:
        file_pointer.write(str(accuracy)+' ')
    file_pointer.write('\n')
    file_pointer.write('END'+'\n')
#********************************************************

In [13]:
def create_dict(start_time, end_time):
    #make it in HH:MM:SS format
    start_time = start_time.split('.')[0]
    start =datetime.strptime(start_time, '%H:%M:%S')
    end_time = end_time.split('.')[0]
    end = datetime.strptime(end_time, '%H:%M:%S')
    time_diff = end - start
    time_list = [start + timedelta(seconds=x) for x in range(time_diff.seconds + 1)]
    time_dict = {time.strftime('%H:%M:%S'): 0 for time in time_list}
    return time_dict

In [ ]:
def getHEPscores(log_file_path, track_log_file,file_pointer):
    #get the session info
    comm_list = []
    sysmon_list = []
    time_dict = {}
    track_list = []
    #open the log file and get the start and end time
    with open(log_file_path, 'r') as f:
        lines  = f.readlines()[2:]

    start_time = lines[0].split()[0]
    end_time = lines[-1].split()[0]
    f.close()

    # traverse thorugh the log file for sysmon and communication events
    with open(file_path ,'r') as f:
        for line in f:
            line = line.split('\t')

            if(len(line)<=1):
                continue

            if line[2] == 'SYSMON':
                sysmon_list.append(line)

            if line[2] == 'COMMUN' and line[6]!='SELECTED\n':
                comm_list.append(line)
    
    time_dict = create_dict(start_time, end_time)
    getCommScore(comm_list,time_dict,file_pointer)
    getSysmonScore(sysmon_list,time_dict,file_pointer)

    #opne the track log file and get the track events
    with open(track_log_file ,'r') as tf:
        for line in tf:
            temp = line.split(' ')[0]
            if(temp=='OUT:'):
                time = line.split(' ')[2]
                time = time.split('.')[0]
                time_dict[time] = 1
            track_list.append(line)
    
    tf.close()
    getTrackScore(track_list,time_dict,file_pointer)

    #convert time_dict to a csv file with header as Timestamp and State
    with open('time_dict.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Timestamp','State'])
        for key, value in time_dict.items():
            writer.writerow([key, value])
    
    

In [6]:
comm_list = []
sysmon_list = []
start_time = 0
end_time = 0

In [7]:
with open(file_path, 'r') as f:
    lines  = f.readlines()[2:]

start_time = lines[0].split()[0]
end_time = lines[-1].split()[0]
f.close()

In [8]:
# traverse thorugh the log file for sysmon and communication events
with open(file_path ,'r') as f:
    for line in f:
        line = line.split('\t')

        if(len(line)<=1):
            continue

        if line[2] == 'SYSMON':
            sysmon_list.append(line)

        if line[2] == 'COMMUN' and line[6]!='SELECTED\n':
            comm_list.append(line)